# Homework 12
## Mengxiang Jiang

The majority of the code here comes from the [Companion Jupyter notebooks for the book "Deep Learning with Python"](https://github.com/fchollet/deep-learning-with-python-notebooks) by François Chollet, specifically chapter 11.
If the code comes from somewhere else it will have a source linked to the original.\
[Video Link](https://youtu.be/eznAqvv8PaI)

### Preparing the IMDB movie reviews data

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

/bin/bash: /home/habib/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  26.2M      0  0:00:03  0:00:03 --:--:-- 26.3M
/bin/bash: /home/habib/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
!rm -r aclImdb/train/unsup

/bin/bash: /home/habib/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
!cat aclImdb/train/pos/4077_10.txt

/bin/bash: /home/habib/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [8]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Displaying the shapes and dtypes of the first batch**

In [9]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'One night I was waiting for my friends to come back to the apt and "Gymkata" happened to be on; I watched way too much of it. It is indeed hilarious, and horrifying, really. Think about it this way--if in your job you had an idea for something this bad and went on to execute it in as terrible a fashion as this, how long exactly would you last? Not as long as this movie. It\'s a must-see, obviously.', shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


**Vectorizing the data**

In [10]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**Transformer encoder implemented as a subclassed `Layer`**

In [11]:
import tensorflow as tf

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**Using the Transformer encoder for text classification**

In [12]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

**Training and evaluating the Transformer encoder based model**

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "transformer_encoder.keras",
        save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
    )
]
encoder_history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


2023-04-24 16:00:42.679389: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-24 16:00:43.076519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-04-24 16:00:43.470759: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fee34004160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-24 16:00:43.470775: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-04-24 16:00:43.472846: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-24 16:00:43.530652: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of 

625/625 [==============================] - 34s 51ms/step - loss: 0.4914 - accuracy: 0.7700 - val_loss: 0.3888 - val_accuracy: 0.8248
Epoch 2/20
625/625 [==============================] - 19s 31ms/step - loss: 0.3431 - accuracy: 0.8486 - val_loss: 0.3114 - val_accuracy: 0.8696
Epoch 3/20
625/625 [==============================] - 16s 26ms/step - loss: 0.3090 - accuracy: 0.8691 - val_loss: 0.3036 - val_accuracy: 0.8756
Epoch 4/20
625/625 [==============================] - 14s 22ms/step - loss: 0.2770 - accuracy: 0.8839 - val_loss: 0.2908 - val_accuracy: 0.8792
Epoch 5/20
625/625 [==============================] - 14s 22ms/step - loss: 0.2469 - accuracy: 0.8983 - val_loss: 0.3246 - val_accuracy: 0.8640
Epoch 6/20
625/625 [==============================] - 13s 20ms/step - loss: 0.2122 - accuracy: 0.9168 - val_loss: 0.2877 - val_accuracy: 0.8840
Epoch 7/20
625/625 [==============================] - 12s 20ms/step - loss: 0.1814 - accuracy: 0.9300 - val_loss: 0.3030 - val_accuracy: 0.8836
Epo

In [14]:
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 5s 7ms/step - loss: 0.3054 - accuracy: 0.8735
Test acc: 0.873


## Beyond text classification: Sequence-to-sequence learning
### A machine translation example

In [15]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

/bin/bash: /home/habib/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
--2023-04-24 16:04:58--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.111.128, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  4.26MB/s    in 0.6s    

2023-04-24 16:04:59 (4.26 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]

/bin/bash: /home/habib/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [16]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [17]:
import random
print(random.choice(text_pairs))

('I am happiest when I am reading.', '[start] Soy el más feliz cuando leo. [end]')


In [18]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

In [19]:
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Preparing datasets for the translation task**

In [20]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [21]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2023-04-24 16:07:38.417465: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Sequence-to-sequence learning with Transformer
#### The Transformer decoder

In [22]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation
**PositionalEmbedding layer**

In [23]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [24]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [25]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "transformer_translate.keras",
        save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
    )
]

transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
translation_history = transformer.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Epoch 1/30
1302/1302 [==============================] - 37s 26ms/step - loss: 3.8069 - accuracy: 0.4384 - val_loss: 2.8890 - val_accuracy: 0.5335
Epoch 2/30
1302/1302 [==============================] - 28s 22ms/step - loss: 2.8638 - accuracy: 0.5488 - val_loss: 2.5459 - val_accuracy: 0.5847
Epoch 3/30
1302/1302 [==============================] - 28s 22ms/step - loss: 2.5643 - accuracy: 0.5932 - val_loss: 2.4513 - val_accuracy: 0.6009
Epoch 4/30
1302/1302 [==============================] - 28s 22ms/step - loss: 2.3988 - accuracy: 0.6193 - val_loss: 2.3318 - val_accuracy: 0.6245
Epoch 5/30
1302/1302 [==============================] - 28s 22ms/step - loss: 2.2951 - accuracy: 0.6371 - val_loss: 2.3223 - val_accuracy: 0.6340
Epoch 6/30
1302/1302 [==============================] - 28s 22ms/step - loss: 2.2209 - accuracy: 0.6511 - val_loss: 2.2987 - val_accuracy: 0.6378
Epoch 7/30
1302/1302 [==============================] - 28s 22ms/step - loss: 2.1558 - accuracy: 0.6643 - val_loss: 2.2875 -

In [30]:
transformer.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['english[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 positional_embedding_1 (Positi  (None, None, 256)   3845120     ['spanish[0][0]']          

**Translating new sentences with our Transformer model**

In [27]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom never did that here.
[start] tom nunca hizo eso [end]
-
Tom has a problem with drug abuse.
[start] tom tiene un problema con la [UNK] [end]
-
I knew you wanted me.
[start] sabía que me querías que yo me quería [end]
-
I don't want to leave.
[start] no quiero salir [end]
-
I haven't heard from her since then.
[start] no la he oído desde entonces [end]
-
There's no more salt.
[start] no queda más sal [end]
-
His creditors are after him.
[start] su profesor está [UNK] [end]
-
Why don't you want to listen to me?
[start] por qué no quieres que me [UNK] [end]
-
I have to warn Tom.
[start] tengo que [UNK] a tom [end]
-
I was watching TV at this time yesterday.
[start] yo estaba viendo televisión a esta noche [end]
-
What do you say to a beer?
[start] qué le digas a decir una cerveza [end]
-
You're not sick.
[start] no estás enfermo [end]
-
Tom stuck his tongue out at Mary.
[start] tom estaba comiendo en la lengua que mary [end]
-
They must be Americans.
[start] ellos debe ser [UNK] [end

In [28]:
sentence1 = "I am a very good student."
spanish_sentence1 = decode_sequence(sentence1)
sentence2a = "Artificial intelligence is a great field"
spanish_sentence2a = decode_sequence(sentence2a)
sentence2b = "it has the potential to shape the future"
spanish_sentence2b = decode_sequence(sentence2b)
sentence3a = "Winston Churchill said"
spanish_sentence3a = decode_sequence(sentence3a)
sentence3b = "success is not final"
spanish_sentence3b = decode_sequence(sentence3b)
sentence3c = "failure is not fatal"
spanish_sentence3c = decode_sequence(sentence3c)
sentence3d = "it is the courage to continue that counts"
spanish_sentence3d = decode_sequence(sentence3d)

In [29]:
print(spanish_sentence1)
print(spanish_sentence2a)
print(spanish_sentence2b)
print(spanish_sentence3a)
print(spanish_sentence3b)
print(spanish_sentence3c)
print(spanish_sentence3d)

[start] soy muy buena estudiante [end]
[start] el [UNK] es una gran país [end]
[start] es el que el camino hasta el futuro [end]
[start] [UNK] dijo que se [UNK] [end]
[start] el éxito no es perfecto [end]
[start] no tiene éxito no [UNK] [end]
[start] es inútil que el valor de eso [UNK] [end]


**Google Translate Back to English**

```
I am a very good student
the [UNK] is a great country
Is the one the way to the future
[UNK] said to be [UNK]
success is not perfect
not successful no [UNK]
it is useless that the value of that [UNK]
```